In [17]:
import gym

env = gym.envs.make("CartPole-v1")

In [18]:
env.reset()

(array([-0.04820433,  0.02400856, -0.03861728,  0.00659723], dtype=float32),
 {})

In [12]:
x = 3

In [14]:
x = x * x

In [15]:
print(x)

9
